In [1]:
import pandas as pd
import csv
import json
import time
import botometer
import math

df = pd.read_csv('./res.csv')

df

c:\users\86173\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (0,2,22,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tweet_id,text,author_id,name,username,in_reply_to_user_id,geo,conversation_id,created_at,lang,...,description,location,profile_image_url,protected,followers_count,following_count,listed_count,tweet_count,verified,url
0,1081698475815841792,I wish our cowardly #LPC #NDP &amp; #CPC polit...,246667561,Pete Quily,pqpolitics,None,None,1.081698e+18,2019-01-05T23:46:28.000Z,en,...,Political & news acct. of information omnivore...,"Vancouver, BC",https://pbs.twimg.com/profile_images/508950241...,False,6507.0,7114.0,462.0,608326.0,False,https://t.co/FJ4NnVDqjF
1,1081698433453309952,RT @joannachiu: Full statement from Global Aff...,1484063959,Paul Macgregor,paulmacgregorCH,None,None,1.081698e+18,2019-01-05T23:46:18.000Z,en,...,"History: Chinese Australian, Chinese diaspora,...","Victoria, Australia",https://pbs.twimg.com/profile_images/378800000...,False,1240.0,985.0,25.0,13912.0,False,http://t.co/0qJZzYsYGX
2,1081690372957487105,Ramifications of the Meng Wanzhou Case (2)\nCC...,977550598118649856,TRUTH MEDIA,TruthMediaEN,None,None,1.081690e+18,2019-01-05T23:14:16.000Z,en,...,TRUTH IS POWER.,None,https://pbs.twimg.com/profile_images/136442062...,False,433.0,90.0,2.0,1528.0,False,None
3,1081675873202229248,RT @joannachiu: Star Editorial: Senators and M...,2823722803,Winston Szeto,winstonszeto,None,None,1.081676e+18,2019-01-05T22:16:39.000Z,en,...,Journalist @cbcnewsbc. Views my own. He/him. 🏳...,"Kelowna, British Columbia 🇨🇦",https://pbs.twimg.com/profile_images/145685884...,False,1312.0,4977.0,16.0,8600.0,False,https://t.co/qjbtvt59GP
4,1081663244576612352,#giveusdata #cdnpoli #bcpoli #vanpoli #YVRRE #...,295913670,Victor Y Wong,VictorY_Wong,295913670,None,1.058407e+18,2019-01-05T21:26:28.000Z,en,...,Victor Wong\n#cdnpoli #cdnimm #onpoli #bcpoli ...,None,https://pbs.twimg.com/profile_images/146001378...,False,406.0,158.0,22.0,59258.0,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34872,1437087656886759428,RT @Simon_A_Wood: The REAL Reason #MengWanzhou...,182663887,#FreePalestine,Simon_A_Wood,None,None,1.437088e+18,2021-09-12T16:16:11.000Z,en,...,Made #AdventuresInAmerica on YouTube.\n#FreeJu...,Australia,https://pbs.twimg.com/profile_images/111998156...,False,7303.0,7109.0,150.0,322228.0,False,https://t.co/ibUqeLoCWz
34873,1437087635181223944,The REAL Reason #MengWanzhou Was Arrested? \n\...,182663887,#FreePalestine,Simon_A_Wood,None,None,1.437088e+18,2021-09-12T16:16:06.000Z,en,...,Made #AdventuresInAmerica on YouTube.\n#FreeJu...,Australia,https://pbs.twimg.com/profile_images/111998156...,False,7303.0,7109.0,150.0,322228.0,False,https://t.co/ibUqeLoCWz
34874,1437055937731182592,Canada needs a strong government to stand up t...,1290390627218108416,Lucy Crawford,LucyCrawford88,None,None,1.437056e+18,2021-09-12T14:10:09.000Z,en,...,"Deep appreciation for Canada, the last heaven ...",Canada,https://pbs.twimg.com/profile_images/129039082...,False,89.0,156.0,0.0,942.0,False,None
34875,1437028870788505600,RT @jenniferatntd: #Iran no good for not being...,1382195314929532928,ⅥⅧ,V0rlNNrO1RqPDnW,None,None,1.437029e+18,2021-09-12T12:22:36.000Z,en,...,None,None,https://abs.twimg.com/sticky/default_profile_i...,False,1.0,41.0,0.0,39.0,False,None


In [2]:
# BotometerAPI配置

# botometerAPI
rapidapi_key = "413fb65c04msh662dbc4d886621ap1aaed3jsnc6ce77e69865"
# Twitter API的密钥
twitter_app_auth = {
    'consumer_key': '5qDFi9HDTPYw9NoXGbgOmDgjH',
    'consumer_secret': 'ALHGUKpEvvShHLuA82zTAT85dbrHNKdrkXMfsoQ41WNpcuxuOE',
    'access_token': '1418747495828451331-37WC8wMOkpaozZAEl05okSFCV0uZ3m',
    'access_token_secret': '6CVAWOqHoqshkenxbsFv18ayx3H7d74Jtg3mnhs21DzWH',
  }
  
# 创建Botometer对象
blt_twitter = botometer.BotometerLite(rapidapi_key=rapidapi_key, **twitter_app_auth)

In [7]:
# 数据去重去空（很重要）

df = df.dropna(subset=['author_id'])
df = df.drop_duplicates('author_id')
userList = list(df['author_id'])
count = len(userList) / 100
count = math.floor(count) # 计算请求次数
count

15842


158

In [8]:
# 准备写入文件
resFile = open('./boto.csv', "a", newline="", encoding='utf-8')
csvWriter = csv.writer(resFile)
csvWriter.writerow(['botscore', 'tweet_id', 'user_id'])
resFile.close()

In [5]:
def writeToCsv(file, row):
    file = open(file, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(file)
    csvWriter.writerow(row)
    file.close()

In [6]:
def run():
    for i in range(0, count):
        print(f'正在爬取第{i}轮...')
        tempList = userList[i*100:(i+1)*100]
        blt_scores = blt_twitter.check_accounts_from_user_ids(tempList)
        tempStr = json.dumps(blt_scores)
        f = open(f'./archive1/new_json{i}.json', 'w')
        f.write(tempStr)
        f.close()
        for item in blt_scores:
            botscore = item['botscore']
            tweet_id = item['tweet_id'] if item['tweet_id'] else 'None'
            user_id = item['user_id']
            tempRow = [botscore, tweet_id, user_id]
            writeToCsv('./boto.csv', tempRow)
        time.sleep(5)

正在爬取第0轮...
正在爬取第1轮...
正在爬取第2轮...
正在爬取第3轮...
正在爬取第4轮...
正在爬取第5轮...
正在爬取第6轮...
正在爬取第7轮...
正在爬取第8轮...
正在爬取第9轮...
正在爬取第10轮...
正在爬取第11轮...
正在爬取第12轮...
正在爬取第13轮...
正在爬取第14轮...
正在爬取第15轮...
正在爬取第16轮...
正在爬取第17轮...
正在爬取第18轮...
正在爬取第19轮...
正在爬取第20轮...
正在爬取第21轮...
正在爬取第22轮...
正在爬取第23轮...
正在爬取第24轮...
正在爬取第25轮...
正在爬取第26轮...
正在爬取第27轮...
正在爬取第28轮...
正在爬取第29轮...
正在爬取第30轮...
正在爬取第31轮...
正在爬取第32轮...
正在爬取第33轮...
正在爬取第34轮...
正在爬取第35轮...
正在爬取第36轮...
正在爬取第37轮...
正在爬取第38轮...
正在爬取第39轮...
正在爬取第40轮...
正在爬取第41轮...
正在爬取第42轮...
正在爬取第43轮...
正在爬取第44轮...
正在爬取第45轮...
正在爬取第46轮...
正在爬取第47轮...
正在爬取第48轮...
正在爬取第49轮...
正在爬取第50轮...
正在爬取第51轮...
正在爬取第52轮...
正在爬取第53轮...
正在爬取第54轮...
正在爬取第55轮...
正在爬取第56轮...
正在爬取第57轮...
正在爬取第58轮...
正在爬取第59轮...
正在爬取第60轮...
正在爬取第61轮...
正在爬取第62轮...
正在爬取第63轮...
正在爬取第64轮...
正在爬取第65轮...
正在爬取第66轮...
正在爬取第67轮...
正在爬取第68轮...
正在爬取第69轮...
正在爬取第70轮...
正在爬取第71轮...
正在爬取第72轮...
正在爬取第73轮...
正在爬取第74轮...
正在爬取第75轮...
正在爬取第76轮...
正在爬取第77轮...
正在爬取第78轮...
正在爬取第79轮...
正在爬取第80轮...
正在爬取第81轮...
正在爬取第82轮...
正在爬取第83轮...
正在

In [9]:
def last():  
    last = userList[15800:]
    last_score = blt_twitter.check_accounts_from_user_ids(last)
    for item in last_score:
        botscore = item['botscore']
        tweet_id = item['tweet_id'] if item['tweet_id'] else 'None'
        user_id = item['user_id']
        tempRow = [botscore, tweet_id, user_id]
        writeToCsv('./boto.csv', tempRow)